# wiki 인덱스 생성

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [2]:
from pinecone import Pinecone
pinecone_client = Pinecone(api_key=pinecone_api_key)

In [3]:
from pinecone import ServerlessSpec
pinecone_client.create_index(
    name='wiki',
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-te99u8p.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [3]:
wiki_index = pinecone_client.Index('wiki')

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# data load

In [5]:
%pip install -qU datasets

Note: you may need to restart the kernel to use updated packages.


In [5]:
from datasets import load_dataset
data = load_dataset("wikimedia/wikipedia", "20231101.ko", split='train[:100]')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Generating train split: 100%|██████████| 647897/647897 [00:01<00:00, 384928.49 examples/s]


In [6]:
data

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100
})

In [7]:
data[0]

{'id': '5',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으

In [8]:
len(data)

100

In [9]:
for rec in data:
    if len(rec['text']) > 35000:
        print('**')

**
**


In [10]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x00000237EE36A570>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x00000237EFFAD8B0>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap = 20,
    length_function = len, \
    separators = ["\n\n", "\n", " ", ""]  # 자르는 순서
)
splitter

In [15]:
import json
def calculate_metadata_size(metadata):
    return len(json.dumps(metadata, ensure_ascii=False).encode('utf-8'))

In [18]:
from uuid import uuid4
import time
texts = []
metas = []
count = 0
batch_size = 20

In [19]:
for i, sample in enumerate(data):
    full_text = sample['text']  # 전문
    
    # 메타데이터 구성
    metadata = {
        "wiki_id": str(sample['id']),
        "url": sample['url'],
        "title": sample['title']
    }
    
    # 스플리터로 스플릿
    chunks = splitter.split_text(full_text)
    # print(len(chunk))
    
    for i, chunk in enumerate(chunks):
        record = {
            'chunk_id': i,
            'full_text': full_text,
            **metadata
        }
        metadata_size = calculate_metadata_size(record)
        if metadata_size > 35000:
            continue
        
        texts.append(chunk)
        metas.append(record)
        count += 1
        
        # 청크 배치의 임베딩 -> 적재
        if count % batch_size == 0:
            ids = [str(uuid4()) for _ in range(len(texts))]
            embeddings = embedding.embed_documents(texts)  # 20개 청크의 임베딩
            wiki_index.upsert(
                vectors=zip(ids, embeddings, metas),
                namespace='wiki-ns1'
            )
            texts = []
            metas = []
            time.sleep(1)

In [20]:
wiki_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki-ns1': {'vector_count': 1160}},
 'total_vector_count': 1160,
 'vector_type': 'dense'}

# 질의 임베딩

In [21]:
question = ["벨기에는 어디에 있나요?"]
emb_question = embedding.embed_documents(question)

# Retrieval

In [23]:
# pinecone 질의 실행
q_result = wiki_index.query(
    namespace='wiki-ns1',
    vector=emb_question,
    top_k=5,
    include_vector=False,
    include_metadata=True
)

In [24]:
result_ids = [r.id for r in q_result.matches]
result_ids

['bf5d6cb1-321f-4897-b63b-85b98def837c',
 '858b8d3b-15ce-4eb6-a3d5-b94150a2b01c',
 '5cf3ea9d-8627-4a95-809a-9ba9a8511216',
 '1c0d54df-eaf8-44b3-a0fd-7a29661ffc0c',
 'c06ecfbe-4334-45cf-ad1c-c6e5b4ab2c97']

In [25]:
for r in q_result.matches:
    print(r.score, r.metadata)

0.306596786 {'chunk_id': 11.0, 'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하는데 있어서 점점 신뢰할 수 없는 예측 기술이 되어감으로 인하여 필수적인 것이 되었다.\n\n열역학적 정의 \n고전적 열역학에서는 엔트로피 S의 절대적 값은 정의할 수 없고, 대신 그 상대적 변화만 정의한다. 열적 평형을 이뤄 온도가 인 계에 열 를 가하였다고 하자. 이 경우 엔트로피의 증가

# 랭체인의 api로 결과 비교하기

In [29]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=wiki_index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=question[0], k=5, namespace='wiki-ns1')
docs

[Document(id='bf5d6cb1-321f-4897-b63b-85b98def837c', metadata={'chunk_id': 11.0, 'title': '엔트로피', 'url': 'https://ko.wikipedia.org/wiki/%EC%97%94%ED%8A%B8%EB%A1%9C%ED%94%BC', 'wiki_id': '158'}, page_content='엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하

In [30]:
question = [
    "주기율표에서 78번에 해당하는 원소는 무엇인가요?",
    "심해는 해수면에서 몇 meter 아래 부터인가요?",
    "행복은 무엇인가요?"
]

In [33]:
for q in question:
    emb_q = embedding.embed_query(f"query: {q}")
    
    q_result = wiki_index.query(
        namespace='wiki-ns1',
        vector=emb_q,
        top_k=5,
        include_vector=False,
        include_metadata=True
    )
    
    for r in q_result.matches:
        print(r.score, r.metadata)
    print("-----------")

0.495598793 {'chunk_id': 16.0, 'full_text': '주기율표(週期律表, , ) 또는 주기표(週期表)는 원소를 구분하기 쉽게 성질에 따라 배열한 표로, 러시아의 드미트리 멘델레예프가 처음 제안했다. 1915년 헨리 모즐리는 멘델레예프의 주기율표를 개량시켜서 원자번호순으로 배열했는데, 이는 현대의 원소 주기율표와 유사하다. 원자 번호가 커짐에 따라 성질이 비슷한 원소가 주기적으로 나타나는 성질인 주기성을 기준으로 원소들을 배열하였다. 주기율표의 가로행은 주기라 부르고, 세로열은 족이라 부른다. 주기마다 같은 성질의 원소가 반복적으로 나타나기 때문에, 같은 족의 원소들은 서로 유사한 화학적 특성을 보인다. 전자를 가지고 있으려 하는 비금속성은 대체로 오른쪽이 더 높으며, 반대로 전자를 주려고 하는 금속성은 대체로 왼쪽이 더 높다. 이러한 화학적 성질은 각 원소의 전자 배치에 기인한다.\n\n1869년 러시아의 화학자 드미트리 멘델레예프가 원자 질량에 따라 원소의 화학적 성질이 주기적으로 변화하는 것에 착안하여 주기율표를 처음으로 만들었다. 당시에는 모든 원소가 발견되지 않았기 때문에 원소 사이에 공백이 남아있었는데, 멘델레예프는 원소의 주기성에 착안하여 원소를 새로 발견하기도 하였다. 원소의 주기성은 19세기 후반에 사실로 인식되었으며, 원자 번호가 발견되고 20세기 초에 양자역학을 통해 원자의 내부 구조를 탐구하며 재확인되었다. 글렌 T. 시보그가 1945년에 악티늄족이 d-블록 원소가 아닌 f-블록 원소라는 사실을 발견함으로써 현대의 주기율표 틀이 완성되었다.\n\n주기율표는 과학의 발전에 따라 계속 개정되고 있다. 자연계에서는 원자 번호 94까지 존재하는 원소들만 존재하는데, 과학자들은 실험실에서 원자번호 94보다 더 무거운 원소들을 합성하고 있다. 현재에는 118개의 원소들이 알려져 있으며 표의 처음 일곱 주기를 빈틈 없이 채우고 있다. 이 일곱 줄을 넘어서 표가 얼마나 뻗어나갈지, 표의 알려진 부분의 주기율이 언제까지 이어질지는

# Augmentation

In [ ]:
# retrieval
country = '슬로베니아'
query = f"{country}의 여행을 위한 관광지와 문화"
docs = vector_store.similarity_search(query=query, k=5, namespace='wiki-ns1')

In [36]:
# augmentation context
context = ""
for result in docs:
    context += result.page_content

In [ ]:
# generation
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000237F61F07A0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000237F61F29F0>, root_client=<openai.OpenAI object at 0x00000237EB732A20>, root_async_client=<openai.AsyncOpenAI object at 0x00000237F61F3140>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [37]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
                    [
                        ('system', '당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행 일정 수립을 도와줍니다. 제한된 정보라도 최대한 활용하여 도움을 주세요.'),
                        ('human', '저는 {country}로 여행을 계획 중입니다.\n 아래 context에서 {country}관련 정보를 찾아 여행 조언을 해주세요.\n 만약 직접적인 정보가 없다면 수도나 관련 정보를 활용해주세요.\n context : {context}'),
                    ]
                )
chat_template

ChatPromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행 일정 수립을 도와줍니다. 제한된 정보라도 최대한 활용하여 도움을 주세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, template='저는 {country}로 여행을 계획 중입니다.\n 아래 context에서 {country}관련 정보를 찾아 여행 조언을 해주세요.\n 만약 직접적인 정보가 없다면 수도나 관련 정보를 활용해주세요.\n context : {context}'), additional_kwargs={})])

In [38]:
prompt = chat_template.format_messages(country=country, context=context)
print(prompt)

[SystemMessage(content='당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행 일정 수립을 도와줍니다. 제한된 정보라도 최대한 활용하여 도움을 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='저는 슬로베니아로 여행을 계획 중입니다.\n 아래 context에서 슬로베니아관련 정보를 찾아 여행 조언을 해주세요.\n 만약 직접적인 정보가 없다면 수도나 관련 정보를 활용해주세요.\n context : 우크라이나()는 동유럽 국가다. 남쪽과 남동쪽으로는 흑해와 아조프해, 동쪽과 북동쪽으로는 러시아, 북쪽과 북서쪽으로는 벨라루스, 서쪽으로는 폴란드, 슬로바키아, 헝가리, 남서쪽으로는 루마니아, 몰도바와 접한다. 키이우가 수도이며 가장 큰 도시다. 동유럽 평원과 이어져 있으며 기후는 비교적 온화한 편이다. 법적 공용어는 우크라이나어이고, 인구 대부분은 우크라이나어를 사용하지만, 대부분 동부 인구(주로 동부 지역과 동남부 지역, 오데사 지역)는 러시아어 사용자이기도 하다. 주요 도시로는 키이우, 도네츠크, 드니프로, 하르키우, 르비우, 오데사, 자포리자가 있다. 2014년 3월 18일 러시아가 크림반도를 합병함에 따라 행정력이 크림반도에 미치지 못하지만, 국제사회는 대체로 크림반도를 우크라이나의 일부라는 태도를 견지하고 있다.\n\n중세 초 루스 카간국으로부터 키예프 루스로 이어진 우크라이나는 오랫동안 투르크족·몽골족 등 지배를 받았다. 19세기까지 대다수 우크라이나 영토가 러시아 제국에 통합되었고, 나머지 부분은 오스트리아-헝가리 통제 아래 있었다. 우크라이나는 러시아 혁명 후 혼란과 끊임 없는 전쟁 속에서 여러 차례 독립을 시도하여 1917년에 민족국가를 건설했으나, 1922년에 소비에트 연방에 강제 합병되었다. 1923년 소비에트 연방 헌법 적용을 받으며 우크라이나 소비에트 사회주의 공화국이란 이름의 구성국으로 존재했다, 1991년 소련 해체와 함께 독립하였다.\n\n지하

In [ ]:
response = llm.invoke(prompt)

AIMessage(content='슬로베니아에 대한 직접적인 정보는 없지만, 슬로베니아에 대한 여행 조언을 드릴 수 있는 몇 가지 팁을 제안합니다. 슬로베니아는 아름다운 자연경관과 역사적인 도시들로 유명한 나라입니다.\n\n1. **수도 류블랴나 (Ljubljana)**: 슬로베니아의 수도인 류블랴나는 아기자기한 건물과 많은 카페들로 가득찬 매력적인 도시입니다. 류블랴나 성을 방문하고, 그곳에서 도시를 한눈에 내려다 보는 것도 좋습니다.\n\n2. **자연 경관**: 슬로베니아의 자연은 매우 다양하고 아름답습니다. \n   - **블레드 호수 (Lake Bled)**: picturesque한 풍경이 유명합니다. 섬에 있는 교회의 모습과 성채도 매우 인상적입니다.\n   - **프리틀리카 동굴 (Postojna Cave)**: 세계에서 가장 큰 동굴 시스템 중 하나로, 내부 탐방이 가능합니다.\n   - **Triglav 국립공원**: 등산과 하이킹을 즐길 수 있는 멋진 장소로, 트리글라브 산이 환상적인 경관을 제공합니다.\n\n3. **음식 경험**: 슬로베니아의 전통 음식인 **졸리차비나 (Žlikrofi)**(감자 만두)와 **사르마 (Sarma)**(양배추 속을 채운 요리) 등을 꼭 시도해 보세요. 또한 슬로베니아 포도주는 지역 특산품으로 유명하니, 와인 테이스팅도 추천드립니다.\n\n4. **문화 체험**: 슬로베니아는 풍부한 전통 문화와 역사적 유격을 가진 나라입니다. 류블랴나와 같은 도시에서 다양한 박물관을 탐방하거나, 지역 축제에 참여해보는 것도 좋은 경험이 될 것입니다.\n\n5. **교통**: 슬로베니아는 소국이기 때문에 대중교통이 잘 되어 있습니다. 기차나 버스를 이용해 도시 간 이동이 편리합니다.\n\n이러한 정보를 바탕으로 슬로베니아 여행의 재미와 아름다움을 최대한 느끼실 수 있을 것입니다. 안전하고 즐거운 여행 되세요!', additional_kwargs={'refusal': None}, response_metadata={'token_u

In [40]:
print(response.content)

슬로베니아에 대한 직접적인 정보는 없지만, 슬로베니아에 대한 여행 조언을 드릴 수 있는 몇 가지 팁을 제안합니다. 슬로베니아는 아름다운 자연경관과 역사적인 도시들로 유명한 나라입니다.

1. **수도 류블랴나 (Ljubljana)**: 슬로베니아의 수도인 류블랴나는 아기자기한 건물과 많은 카페들로 가득찬 매력적인 도시입니다. 류블랴나 성을 방문하고, 그곳에서 도시를 한눈에 내려다 보는 것도 좋습니다.

2. **자연 경관**: 슬로베니아의 자연은 매우 다양하고 아름답습니다. 
   - **블레드 호수 (Lake Bled)**: picturesque한 풍경이 유명합니다. 섬에 있는 교회의 모습과 성채도 매우 인상적입니다.
   - **프리틀리카 동굴 (Postojna Cave)**: 세계에서 가장 큰 동굴 시스템 중 하나로, 내부 탐방이 가능합니다.
   - **Triglav 국립공원**: 등산과 하이킹을 즐길 수 있는 멋진 장소로, 트리글라브 산이 환상적인 경관을 제공합니다.

3. **음식 경험**: 슬로베니아의 전통 음식인 **졸리차비나 (Žlikrofi)**(감자 만두)와 **사르마 (Sarma)**(양배추 속을 채운 요리) 등을 꼭 시도해 보세요. 또한 슬로베니아 포도주는 지역 특산품으로 유명하니, 와인 테이스팅도 추천드립니다.

4. **문화 체험**: 슬로베니아는 풍부한 전통 문화와 역사적 유격을 가진 나라입니다. 류블랴나와 같은 도시에서 다양한 박물관을 탐방하거나, 지역 축제에 참여해보는 것도 좋은 경험이 될 것입니다.

5. **교통**: 슬로베니아는 소국이기 때문에 대중교통이 잘 되어 있습니다. 기차나 버스를 이용해 도시 간 이동이 편리합니다.

이러한 정보를 바탕으로 슬로베니아 여행의 재미와 아름다움을 최대한 느끼실 수 있을 것입니다. 안전하고 즐거운 여행 되세요!
